In [1]:
#调试开关
import logging
#logging.basicConfig(level=logging.INFO,format="%(filename)s[line:%(lineno)d] %(levelname)s %(message)s")
#logger=logging.getLogger(__name__)
import sys
sys.path.append("../AdvBox/")
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.utils.data.dataloader as Data
from advbox.adversary import Adversary
from advbox.attacks.gradient_method import FGSM
from advbox.models.pytorch import PytorchModel

In [2]:
# 训练mnist的cnn网络定义
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(   # (1,28,28)   
            # in_channels:输入的通道数， out_channels：卷积核数量， kernel_size：卷积核大小， stride：步长
            # stride=1时， padding=(kernel_size-1)/2， 图片长宽不变
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2),    # (16,28,28)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),    # (16,14,14)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),    # (32,14,14)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),    # （32,7,7）
        )
        self.fc1 = nn.Sequential(
            nn.Linear(32*7*7, 500),
            nn.ReLU()
        )
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)    # (batch,32,7,7)
        x = x.view(x.size(0), -1)    # (batch,32*7*7)
        x = self.fc1(x)
        output = self.fc2(x)
        return output

In [3]:
TOTAL_NUM = 10000
pretrained_model = "model/mnist_model_dict.pth"
loss_func = torch.nn.CrossEntropyLoss()

#使用MNIST测试数据集 随机挑选TOTAL_NUM个
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/home/shenchenkai/data/mnist_pytorch/',
                   train=False,
                   download=False,
                   transform=transforms.Compose([transforms.ToTensor(),])),
    batch_size=1,
    shuffle=True)

# Define what device we are using
logging.info("CUDA Available: {}".format(torch.cuda.is_available()))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the network
model = CNN().to(device)

# Load the pretrained model
model.load_state_dict(torch.load(pretrained_model, map_location='cpu'))

# Set the model in evaluation mode. In this case this is for the Dropout layers
model.eval()

# advbox demo
m = PytorchModel(model, loss_func, (0, 1), channel_axis=1)

#实例化FGSM
attack = FGSM(m)
#设置攻击步长为0.1
attack_config = {"epsilons": 0.1}

# use test data to generate adversarial examples
total_count = 0
fooling_count = 0

for i, data in enumerate(test_loader):
    inputs, labels = data
    inputs, labels = inputs.numpy(), labels.numpy()

    total_count += 1
    adversary = Adversary(inputs, labels[0])

    # FGSM non-targeted attack
    adversary = attack(adversary, **attack_config)

    if adversary.is_successful():
        fooling_count += 1
        print(
            'attack success, original_label=%d, adversarial_label=%d, count=%d'
            % (labels, adversary.adversarial_label, total_count))

    else:
        print('attack failed, original_label=%d, count=%d' %
              (labels, total_count))

    if total_count >= TOTAL_NUM:
        print(
            "[TEST_DATASET]: fooling_count=%d, total_count=%d, fooling_rate=%f"
            % (fooling_count, total_count, float(fooling_count) / total_count))
        break
print("fgsm attack done")

cuda
attack success, original_label=3, adversarial_label=9, count=1
attack success, original_label=5, adversarial_label=3, count=2
attack success, original_label=7, adversarial_label=9, count=3
attack success, original_label=8, adversarial_label=5, count=4
attack success, original_label=1, adversarial_label=8, count=5
attack success, original_label=9, adversarial_label=4, count=6
attack success, original_label=7, adversarial_label=3, count=7
attack success, original_label=7, adversarial_label=3, count=8
attack success, original_label=4, adversarial_label=9, count=9
attack success, original_label=3, adversarial_label=9, count=10
attack success, original_label=2, adversarial_label=0, count=11
attack success, original_label=0, adversarial_label=9, count=12
attack success, original_label=1, adversarial_label=8, count=13
attack success, original_label=0, adversarial_label=6, count=14
attack success, original_label=8, adversarial_label=3, count=15
attack success, original_label=6, adversaria

attack success, original_label=0, adversarial_label=9, count=136
attack success, original_label=7, adversarial_label=9, count=137
attack success, original_label=9, adversarial_label=4, count=138
attack success, original_label=3, adversarial_label=8, count=139
attack success, original_label=1, adversarial_label=4, count=140
attack success, original_label=1, adversarial_label=8, count=141
attack success, original_label=2, adversarial_label=9, count=142
attack success, original_label=0, adversarial_label=9, count=143
attack success, original_label=3, adversarial_label=8, count=144
attack success, original_label=7, adversarial_label=8, count=145
attack success, original_label=0, adversarial_label=9, count=146
attack success, original_label=8, adversarial_label=0, count=147
attack success, original_label=4, adversarial_label=2, count=148
attack success, original_label=0, adversarial_label=9, count=149
attack success, original_label=1, adversarial_label=8, count=150
attack success, original_

attack success, original_label=8, adversarial_label=3, count=270
attack success, original_label=1, adversarial_label=8, count=271
attack success, original_label=2, adversarial_label=8, count=272
attack success, original_label=5, adversarial_label=9, count=273
attack success, original_label=2, adversarial_label=1, count=274
attack success, original_label=7, adversarial_label=3, count=275
attack success, original_label=6, adversarial_label=8, count=276
attack success, original_label=6, adversarial_label=8, count=277
attack success, original_label=3, adversarial_label=5, count=278
attack success, original_label=0, adversarial_label=6, count=279
attack success, original_label=6, adversarial_label=8, count=280
attack success, original_label=0, adversarial_label=8, count=281
attack success, original_label=6, adversarial_label=9, count=282
attack success, original_label=8, adversarial_label=6, count=283
attack success, original_label=0, adversarial_label=9, count=284
attack success, original_

attack success, original_label=0, adversarial_label=9, count=402
attack success, original_label=5, adversarial_label=8, count=403
attack success, original_label=1, adversarial_label=8, count=404
attack success, original_label=8, adversarial_label=5, count=405
attack success, original_label=1, adversarial_label=8, count=406
attack success, original_label=3, adversarial_label=8, count=407
attack success, original_label=4, adversarial_label=8, count=408
attack success, original_label=7, adversarial_label=9, count=409
attack success, original_label=3, adversarial_label=8, count=410
attack success, original_label=4, adversarial_label=9, count=411
attack success, original_label=8, adversarial_label=3, count=412
attack success, original_label=0, adversarial_label=8, count=413
attack success, original_label=1, adversarial_label=8, count=414
attack success, original_label=8, adversarial_label=4, count=415
attack success, original_label=6, adversarial_label=4, count=416
attack success, original_

attack success, original_label=2, adversarial_label=1, count=544
attack success, original_label=9, adversarial_label=4, count=545
attack success, original_label=1, adversarial_label=8, count=546
attack success, original_label=2, adversarial_label=3, count=547
attack success, original_label=4, adversarial_label=9, count=548
attack success, original_label=0, adversarial_label=9, count=549
attack success, original_label=0, adversarial_label=6, count=550
attack success, original_label=2, adversarial_label=3, count=551
attack success, original_label=2, adversarial_label=0, count=552
attack success, original_label=1, adversarial_label=8, count=553
attack success, original_label=8, adversarial_label=5, count=554
attack success, original_label=6, adversarial_label=4, count=555
attack success, original_label=1, adversarial_label=8, count=556
attack success, original_label=9, adversarial_label=5, count=557
attack success, original_label=8, adversarial_label=9, count=558
attack success, original_

...

attack success, original_label=4, adversarial_label=8, count=9333
attack success, original_label=3, adversarial_label=8, count=9334
attack success, original_label=4, adversarial_label=8, count=9335
attack success, original_label=6, adversarial_label=4, count=9336
attack success, original_label=8, adversarial_label=4, count=9337
attack success, original_label=0, adversarial_label=8, count=9338
attack success, original_label=9, adversarial_label=4, count=9339
attack success, original_label=3, adversarial_label=8, count=9340
attack success, original_label=2, adversarial_label=1, count=9341
attack success, original_label=0, adversarial_label=8, count=9342
attack success, original_label=3, adversarial_label=5, count=9343
attack success, original_label=1, adversarial_label=8, count=9344
attack success, original_label=2, adversarial_label=3, count=9345
attack success, original_label=2, adversarial_label=8, count=9346
attack success, original_label=8, adversarial_label=3, count=9347
attack suc

attack success, original_label=6, adversarial_label=8, count=9470
attack success, original_label=9, adversarial_label=8, count=9471
attack success, original_label=8, adversarial_label=3, count=9472
attack success, original_label=7, adversarial_label=3, count=9473
attack success, original_label=2, adversarial_label=3, count=9474
attack success, original_label=9, adversarial_label=8, count=9475
attack success, original_label=3, adversarial_label=8, count=9476
attack success, original_label=6, adversarial_label=0, count=9477
attack success, original_label=3, adversarial_label=9, count=9478
attack success, original_label=1, adversarial_label=8, count=9479
attack success, original_label=0, adversarial_label=8, count=9480
attack success, original_label=8, adversarial_label=3, count=9481
attack success, original_label=4, adversarial_label=9, count=9482
attack success, original_label=4, adversarial_label=9, count=9483
attack success, original_label=4, adversarial_label=8, count=9484
attack suc

attack success, original_label=1, adversarial_label=8, count=9611
attack success, original_label=8, adversarial_label=5, count=9612
attack success, original_label=3, adversarial_label=8, count=9613
attack success, original_label=7, adversarial_label=9, count=9614
attack success, original_label=9, adversarial_label=4, count=9615
attack success, original_label=2, adversarial_label=0, count=9616
attack success, original_label=5, adversarial_label=9, count=9617
attack success, original_label=8, adversarial_label=0, count=9618
attack success, original_label=9, adversarial_label=4, count=9619
attack success, original_label=5, adversarial_label=8, count=9620
attack success, original_label=3, adversarial_label=5, count=9621
attack success, original_label=2, adversarial_label=3, count=9622
attack success, original_label=7, adversarial_label=3, count=9623
attack success, original_label=0, adversarial_label=6, count=9624
attack success, original_label=5, adversarial_label=6, count=9625
attack suc

attack success, original_label=1, adversarial_label=8, count=9756
attack success, original_label=6, adversarial_label=4, count=9757
attack success, original_label=9, adversarial_label=8, count=9758
attack success, original_label=7, adversarial_label=9, count=9759
attack success, original_label=2, adversarial_label=3, count=9760
attack success, original_label=4, adversarial_label=9, count=9761
attack success, original_label=4, adversarial_label=8, count=9762
attack success, original_label=7, adversarial_label=3, count=9763
attack success, original_label=3, adversarial_label=5, count=9764
attack success, original_label=1, adversarial_label=8, count=9765
attack success, original_label=2, adversarial_label=8, count=9766
attack success, original_label=4, adversarial_label=9, count=9767
attack success, original_label=7, adversarial_label=3, count=9768
attack success, original_label=3, adversarial_label=9, count=9769
attack success, original_label=6, adversarial_label=4, count=9770
attack suc

attack success, original_label=1, adversarial_label=8, count=9900
attack success, original_label=1, adversarial_label=4, count=9901
attack success, original_label=7, adversarial_label=9, count=9902
attack success, original_label=3, adversarial_label=5, count=9903
attack success, original_label=9, adversarial_label=8, count=9904
attack success, original_label=2, adversarial_label=8, count=9905
attack success, original_label=0, adversarial_label=9, count=9906
attack success, original_label=0, adversarial_label=9, count=9907
attack success, original_label=0, adversarial_label=9, count=9908
attack success, original_label=9, adversarial_label=4, count=9909
attack success, original_label=9, adversarial_label=8, count=9910
attack success, original_label=1, adversarial_label=8, count=9911
attack success, original_label=5, adversarial_label=3, count=9912
attack success, original_label=1, adversarial_label=8, count=9913
attack success, original_label=9, adversarial_label=4, count=9914
attack suc